In [ ]:
def sample():
    rng = np.random
    pt = dict(
        n_components=2,
        perplexity=rng.choice((2, 5, 10, 30, 50, 100)),
        early_exaggeration=rng.randint(1, 10),
        learning_rate=rng.randint(100, 1000), 
        n_iter=5000,
        n_iter_without_progress=500, 
        min_grad_norm=0, 
        metric='euclidean', 
        init=rng.choice(('pca', 'random')),
        verbose=0, 
        random_state=None,
        method='barnes_hut', 
        angle=0.5
    )
    return pt

def run(pt):
    points_ = points.reshape((points.shape[0], -1))
    return TSNE(**pt).fit(points_)

jobs = [sample() for _ in range(100)]
models = Parallel(n_jobs=10, verbose=1)(delayed(run)(j) for j in jobs)
kl = map(lambda m:m.kl_divergence_, models)

In [ ]:
from collections import defaultdict
perplexities = defaultdict(list)
for m in models:
    perplexities[m.perplexity].append(m)
fig, ax = plt.subplots(1, len(perplexities), figsize=(25, 5))
i = 0
for perp in sorted(perplexities.keys()):
    tsne = min(perplexities.get(perp), key=lambda model:model.kl_divergence_)
    embedding = tsne.embedding_
    ax[i].scatter(embedding[:, 0], embedding[:, 1])
    ax[i].set_title('perplexity {}'.format(perp))
    i += 1
plt.show()

In [ ]:
tsne = min(perplexities[10], key=lambda m:m.kl_divergence_)
embedding = tsne.embedding_

In [ ]:
from tools.viz.image_scatter import image_scatter
pts = points.reshape((points.shape[0], 28, 28)).copy()
pts = pts[:, :, :, None] * np.ones((1, 1, 1, 3))
pts[0:10, :, :, 1:]=0
img = image_scatter(embedding, pts, 20, 500)
imsave('imgs/out.png', img)
Image('imgs/out.png')